In [1]:
#import thư viện
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv('datasets_motels.csv')
df.head()

,id_motels,tennhatro,diachi,toalac,danhgia,gia,dientich,noidung,luotxem
0,0,Cho Thuê Phòng Trọ Có Sẵn Nội Thất,"Địa chỉ: Hẻm 278, đường Tầm Vu",Ninh Kiều,5.0,2700000,DT: 27.00 m²,Mùa nhập học đến rồi_Các bạn sinh viên đã tìm ...,78796
1,1,Cho thuê phòng trọ mới thoáng mát,"Địa chỉ: 156b1. Đường Nguyễn Tri Phương, p An ...",Ninh Kiều,1.2,1000000,DT: 24.00 m²,"Cho thuê phòng trọ đường Nguyễn Tri Phương, p...",1891
2,2,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...","Địa chỉ: 360 Tầm Vu , phường Hưng Lợi , quận N...",Ninh Kiều,2.7,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",13095
3,3,"NHÀ TRỌ ĐƯỜNG TẦM VU , GẦN NHÀ HÀNG MÊKONG , C...","Địa chỉ: 360A ,ĐƯỜNG TẦM VU , PHƯỜNG HƯNG LỢI ...",Ninh Kiều,3.0,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",21811
4,4,"Phòng trọ, nhà nghỉ Hoàng Phương Đông cần cho ...",Địa chỉ: 14R Trần Văn Sắc,Ninh Kiều,1.5,2000000,DT: 25.00 m²,Cho thuê phòng trọ mới sạch đẹp trang thiết bị...,8011


In [3]:
def sentiment(gia):
  if gia in range(0,1000000):
    return 'Dưới 1 triệu'
  elif gia in range(1000001,2000000):
    return '1 triệu - 2 triệu'
  else:
    return 'Trên 2 triệu'

In [4]:
df['gia1'] = df['gia'].apply(sentiment)
df

,id_motels,tennhatro,diachi,toalac,danhgia,gia,dientich,noidung,luotxem,gia1
0,0,Cho Thuê Phòng Trọ Có Sẵn Nội Thất,"Địa chỉ: Hẻm 278, đường Tầm Vu",Ninh Kiều,5.0,2700000,DT: 27.00 m²,Mùa nhập học đến rồi_Các bạn sinh viên đã tìm ...,78796,Trên 2 triệu
1,1,Cho thuê phòng trọ mới thoáng mát,"Địa chỉ: 156b1. Đường Nguyễn Tri Phương, p An ...",Ninh Kiều,1.2,1000000,DT: 24.00 m²,"Cho thuê phòng trọ đường Nguyễn Tri Phương, p...",1891,Trên 2 triệu
2,2,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...","Địa chỉ: 360 Tầm Vu , phường Hưng Lợi , quận N...",Ninh Kiều,2.7,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",13095,1 triệu - 2 triệu
3,3,"NHÀ TRỌ ĐƯỜNG TẦM VU , GẦN NHÀ HÀNG MÊKONG , C...","Địa chỉ: 360A ,ĐƯỜNG TẦM VU , PHƯỜNG HƯNG LỢI ...",Ninh Kiều,3.0,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",21811,1 triệu - 2 triệu
4,4,"Phòng trọ, nhà nghỉ Hoàng Phương Đông cần cho ...",Địa chỉ: 14R Trần Văn Sắc,Ninh Kiều,1.5,2000000,DT: 25.00 m²,Cho thuê phòng trọ mới sạch đẹp trang thiết bị...,8011,Trên 2 triệu
...,...,...,...,...,...,...,...,...,...,...
403,403,NHÀ TRO MINH LUAN,"Địa chỉ: Cầu Lê Bình, Thường Thạnh, Cái Răng, ...",Cái Răng,5.0,700000,DT: 32.00 m²,Cho thuê phòng trọ mới sạch đẹp trang thiết bị...,34712,Dưới 1 triệu
404,404,Nhà trọ Ngọc Huyền,"Địa chỉ: 2PJW+FWF, Phường An Khánh, Ninh Kiều,...",Ninh Kiều,0.0,1000000,DT: 45.00 m²,Nhà trọ - Thoáng mát - An ninh - Wc trong - Wi...,123569,Trên 2 triệu
405,405,Nhà Trọ Minh Quân,"Địa chỉ: 25 Đường số 1, Khu Nhà Ở Cán Bộ Giáo ...",Ninh Kiều,0.0,1500000,DT: 25.00 m²,"Nhà trọ cao ráo, thoáng mát, sạch đẹp, thiết k...",0,1 triệu - 2 triệu
406,406,Phòng Trọ Sinh Viên\r,"Địa chỉ: K4 Nguyễn Ngọc Trai, An Phú, Ninh Kiề...",Ninh Kiều,0.0,1000000,DT: 20.00 m²,"Phòng trọ đẹp, sạch sẽ, thoáng mát, yên tĩnh, ...",1156,Trên 2 triệu


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_motels  408 non-null    int64  
 1   tennhatro  408 non-null    object 
 2   diachi     407 non-null    object 
 3   toalac     408 non-null    object 
 4   danhgia    408 non-null    float64
 5   gia        408 non-null    int64  
 6   dientich   408 non-null    object 
 7   noidung    408 non-null    object 
 8   luotxem    408 non-null    int64  
 9   gia1       408 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 32.0+ KB


In [6]:
tfidf = TfidfVectorizer()
df['gia1'] = df['gia1'].fillna('')
#Xây dựng ma trận TF-IDF bắt buộc bằng cách áp dụng phương pháp fit_transform trên tính năng tổng quan
overview_matrix = tfidf.fit_transform(df['gia1'])
#Xuất ra hình dạng của tfidf_matrix
overview_matrix.shape

(408, 3)

In [7]:
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

array([[1.        , 1.        , 0.49036696, ..., 0.49036696, 1.        ,
        0.49036696],
       [1.        , 1.        , 0.49036696, ..., 0.49036696, 1.        ,
        0.49036696],
       [0.49036696, 0.49036696, 1.        , ..., 1.        , 0.49036696,
        1.        ],
       ...,
       [0.49036696, 0.49036696, 1.        , ..., 1.        , 0.49036696,
        1.        ],
       [1.        , 1.        , 0.49036696, ..., 0.49036696, 1.        ,
        0.49036696],
       [0.49036696, 0.49036696, 1.        , ..., 1.        , 0.49036696,
        1.        ]])

In [8]:
#motel index mapping
mapping = pd.Series(df.index,index = df['gia1'])
mapping

gia1
Trên 2 triệu           0
Trên 2 triệu           1
1 triệu - 2 triệu      2
1 triệu - 2 triệu      3
Trên 2 triệu           4
                    ... 
Dưới 1 triệu         403
Trên 2 triệu         404
1 triệu - 2 triệu    405
Trên 2 triệu         406
1 triệu - 2 triệu    407
Length: 408, dtype: int64

In [11]:
def recommend_motel_based_on_plot(motel_input):
  motel_index = mapping[motel_input]
  #nhận các giá trị tương tự với các nhà trọ khác
  #điểm tương tự là danh sách chỉ số và ma trận tương tự
  similarity_score = list(enumerate(similarity_matrix[motel_index]))
  #sắp xếp theo thứ tự giảm dần điểm tương tự của nhà trọ được nhập với tất cả các nhà trọ khác
  similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
  # Nhận điểm của 5 nhà nghỉ giống nhau nhất. Bỏ qua nhà nghỉ đầu tiên.
  similarity_score = similarity_score[0:5]
  #trả lại tên nhà nghỉ bằng cách sử dụng chuỗi ánh xạ
  motel_indices = [i[0] for i in similarity_score]
  return (df[['tennhatro','gia']].iloc[motel_indices])

In [12]:
recommend_motel_based_on_plot('Dưới 1 triệu')

,tennhatro,gia
126,Tìm nam ở ghép sinh viên hoặc đi làm,800000
125,Tim nam ở ghép 700k chưa điện nước,700000
124,Cần nữ ở ghép giá 650k/tháng/người,1200000
123,"Xin ở ghép, do chưa có phòng trọ.",1500000
122,Bình Thủy,800000


In [13]:
def sentiment(luotxem):
  if luotxem in range(0,1000):
    return 'Thấp'
  elif luotxem in range(1001,10000):
    return 'Trung Bình'
  elif luotxem in range(10001,50000):
    return 'Khá'
  elif luotxem in range(50001,70000):
    return 'Tốt'
  else:
    return 'Rất tốt' 

df['luotxem1'] = df['luotxem'].apply(sentiment)
df.head()

,id_motels,tennhatro,diachi,toalac,danhgia,gia,dientich,noidung,luotxem,gia1,luotxem1
0,0,Cho Thuê Phòng Trọ Có Sẵn Nội Thất,"Địa chỉ: Hẻm 278, đường Tầm Vu",Ninh Kiều,5.0,2700000,DT: 27.00 m²,Mùa nhập học đến rồi_Các bạn sinh viên đã tìm ...,78796,Trên 2 triệu,Rất tốt
1,1,Cho thuê phòng trọ mới thoáng mát,"Địa chỉ: 156b1. Đường Nguyễn Tri Phương, p An ...",Ninh Kiều,1.2,1000000,DT: 24.00 m²,"Cho thuê phòng trọ đường Nguyễn Tri Phương, p...",1891,Trên 2 triệu,Trung Bình
2,2,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...","Địa chỉ: 360 Tầm Vu , phường Hưng Lợi , quận N...",Ninh Kiều,2.7,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",13095,1 triệu - 2 triệu,Khá
3,3,"NHÀ TRỌ ĐƯỜNG TẦM VU , GẦN NHÀ HÀNG MÊKONG , C...","Địa chỉ: 360A ,ĐƯỜNG TẦM VU , PHƯỜNG HƯNG LỢI ...",Ninh Kiều,3.0,1200000,DT: 15.00 m²,"NHÀ TRỌ ĐƯỜNG TẦM VU , KẾ TU VIỆN , NHÀ HÀNG M...",21811,1 triệu - 2 triệu,Khá
4,4,"Phòng trọ, nhà nghỉ Hoàng Phương Đông cần cho ...",Địa chỉ: 14R Trần Văn Sắc,Ninh Kiều,1.5,2000000,DT: 25.00 m²,Cho thuê phòng trọ mới sạch đẹp trang thiết bị...,8011,Trên 2 triệu,Trung Bình


In [14]:
tfidf = TfidfVectorizer()
df['luotxem1'] = df['luotxem1'].fillna('')

overview_matrix = tfidf.fit_transform(df['luotxem1'])

overview_matrix.shape

similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix


mapping = pd.Series(df.index,index = df['luotxem1'])


def recommend_motel_based_on_plot(motel_input):
  motel_index = mapping[motel_input]

  similarity_score = list(enumerate(similarity_matrix[motel_index]))

  similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
  
  similarity_score = similarity_score[0:5]
  
  motel_indices = [i[0] for i in similarity_score]
  return (df[['tennhatro','gia']].iloc[motel_indices])

In [15]:
recommend_motel_based_on_plot('Tốt')

,tennhatro,gia
27,Phòng trọ sinh viên giá rẻ địa chỉ 286/13 CMT8,700000
26,Cho thuê phòng trọ đầy đủ tiện nghi tại trung ...,1500000
25,Cho thuê phòng trọ đầy đủ tiện nghi tại trung ...,1500000
24,Cho thuê phòng trọ đầy đủ tiện nghi tại trung ...,1500000
23,Cho thuê phòng trọ sinh viên giá 700. Địa chỉ:...,700000
